In [1]:
import h5py
import numpy as np
filename = '/root/data/models/segmentation-2019-02-26.h5'
f = h5py.File(filename, 'r')

In [2]:
for key in f.keys():
    print(key) #Names of the groups in HDF5 file.

ROI
activation_100
activation_101
activation_102
activation_103
activation_104
activation_105
activation_106
activation_107
activation_108
activation_109
activation_110
activation_111
activation_112
activation_113
activation_114
activation_115
activation_116
activation_117
activation_118
activation_119
activation_120
activation_121
activation_122
activation_123
activation_124
activation_125
activation_126
activation_127
activation_128
activation_129
activation_130
activation_131
activation_132
activation_133
activation_134
activation_135
activation_136
activation_137
activation_138
activation_139
activation_140
activation_141
activation_142
activation_143
activation_145
activation_146
activation_147
activation_148
activation_75
activation_76
activation_77
activation_78
activation_79
activation_80
activation_81
activation_82
activation_83
activation_84
activation_85
activation_86
activation_87
activation_88
activation_89
activation_90
activation_91
activation_92
activation_93
activation

In [4]:
#Get the HDF5 group
group = f["conv1"]

#Checkout what keys are inside that group.
for key in group.keys():
    print(key)

data = group["conv1_1"].value
#Do whatever you want with data

#After you are done
f.close()

conv1_1


AttributeError: 'Group' object has no attribute 'value'

In [10]:
group["conv1_1"]['kernel:0'].value

array([[[[ 2.74530035e-02, -1.13056721e-02, -1.01477057e-02, ...,
           1.47070820e-02,  2.58966293e-02, -1.55841624e-02],
         [ 4.95859273e-02,  1.36352805e-02, -1.13751031e-02, ...,
           4.36610309e-03,  1.36647224e-01, -5.33329835e-03],
         [ 2.82748062e-02, -4.92850738e-03, -1.65555142e-02, ...,
           4.54155449e-03, -9.45690498e-02, -2.46273372e-02]],

        [[ 3.63763832e-02, -2.20013037e-02, -9.81691014e-03, ...,
          -3.30769494e-02, -1.81165766e-02, -2.61035394e-02],
         [ 2.26292070e-02, -6.77385926e-03, -7.92866293e-03, ...,
          -2.49545816e-02,  6.10168763e-02, -3.29514146e-02],
         [ 3.39694619e-02, -1.17062824e-02, -6.05418906e-03, ...,
          -2.43192390e-02, -1.07113114e-02, -4.76608090e-02]],

        [[ 3.26521844e-02, -7.96776265e-03, -1.39790808e-03, ...,
          -1.28367767e-02, -1.40956081e-02, -6.25602677e-02],
         [ 1.85736560e-03,  6.01391913e-03,  9.90595086e-04, ...,
           3.71941645e-03,  3.1045

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

## Create DATASET

### 1 - Split train test

In [ ]:
import json 

In [ ]:
data_path = '/root/data/aquabyte-images/'
coco_file_dir = 'cocofiles'
targets = ['full', 'clear']

In [ ]:
target_coco_files = []
for file in os.listdir(data_path + coco_file_dir):
    count = 0
    for target in targets:
        if target in file:
            count += 1
    if count == len(targets):
        target_coco_files.append(file)  
size_dict = {}
datas = []
for coco_file in target_coco_files:
    with open(data_path + coco_file_dir + '/' + coco_file) as f:
        coco_dict = json.load(f)
        datas.append(coco_dict)
        size = len(coco_dict['annotations'])
        size_dict[size] = data_path + coco_file_dir + '/' + coco_file

In [ ]:
size_dict

In [ ]:
categories_dict = datas[0]['categories']

In [ ]:
test_coco_sizes = [x for x in size_dict.keys() if x < 800]
print('Split is {}'.format(float(sum(test_coco_sizes)) / sum(size_dict.keys())))
test_coco_files = [size_dict[x] for x in test_coco_sizes]
train_coco_files = [size_dict[x] for x in list(set(size_dict.keys()) - set(test_coco_sizes))]

### 2 - Datasets

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import keras

sys.path.insert(0, '../libraries')
from mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.visualize as visualize
from mrcnn.model import log
import mcoco.coco as coco
# import mextra.utils as extra_utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
%config IPCompleter.greedy=True
BASE_DIR = '/root/data/models/erko/mask_rcnn_instance_segmentation'
DATA_DIR = '/root/data/erko/'
WEIGHTS_DIR = os.path.join(BASE_DIR, "weights")
MODEL_DIR = os.path.join(BASE_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(WEIGHTS_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
train_coco_files = ["/root/data/aquabyte-images-resized/cocofiles/coco_body_parts_2018-11-05.json"]

In [ ]:
dataset_train = coco.CocoDataset()
for coco_file in train_coco_files:
    dataset_train.load_coco(coco_file, class_ids=[1, 2, 3, 4, 5, 6, 7, 8])
dataset_train.prepare()
print("Number of train images: {}".format(dataset_train.num_images))

In [ ]:
dataset_validate = coco.CocoDataset()
for coco_file in test_coco_files:
    dataset_validate.load_coco(coco_file, class_ids=[0])
dataset_validate.prepare()
print("Number of train images: {}".format(dataset_validate.num_images))

In [ ]:
# # Load and display random samples# Load  
# image_ids = np.random.choice(dataset_train.image_ids, 4)
# for image_id in image_ids:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Configuration

In [ ]:
image_size = 512
rpn_anchor_template = (1, 2, 4, 8, 16) # anchor sizes in pixels
rpn_anchor_scales = tuple(i * (image_size // 16) for i in rpn_anchor_template)

class FishConfig(Config):
    """Configuration for training on the shapes dataset.
    """
    # name your experiments here
    NAME = "clear_full_512"

    # Train on 1 GPU and 2 images per GPU. Put multiple images on each
    # GPU if the images are small. Batch size is 2 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # background + 3 shapes (triangles, circles, and squares)

    # Use smaller images for faster training. 
    IMAGE_MAX_DIM = image_size
    IMAGE_MIN_DIM = image_size
    
    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = rpn_anchor_scales

    # Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 300
    
config = FishConfig()
# config.display()

## Model

In [ ]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
model.load_weights("/root/data/models/segmentation-2019-02-26.h5")

In [ ]:
np.random.choice(dataset_train.image_ids, 1)

In [ ]:
image = dataset_train.load_image(np.random.choice(dataset_train.image_ids, 1)[0])

In [ ]:
model.

In [ ]:
# inititalize_weights_with = "coco"  # imagenet, coco, or last

# if inititalize_weights_with == "imagenet":
#     model.load_weights(model.get_imagenet_weights(), by_name=True)
    
# elif inititalize_weights_with == "coco":
#     model.load_weights(COCO_MODEL_PATH, by_name=True,
#                        exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
#                                 "mrcnn_bbox", "mrcnn_mask"])
    
# elif inititalize_weights_with == "last":
#     # Load the last model you trained and continue training
#     model.load_weights(model.find_last()[1], by_name=True)

## Training

In [ ]:
# add more callbacks here if necessary
callbacks = [
    keras.callbacks.TensorBoard(log_dir=model.log_dir,
                                histogram_freq=0, write_graph=True, write_images=False),
    keras.callbacks.ModelCheckpoint(model.checkpoint_path,
                                    verbose=0, save_weights_only=True),
]

In [ ]:
model.train(dataset_train, dataset_validate, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=100, # starts from the previous epoch, so only 1 additional is trained 
            layers="all",
            callback_list=callbacks)

## Detection

### 1 - Inference config

In [ ]:
# model_path = '/root.d'
# model_path = '/root/data/models/erko/mask_rcnn_instance_segmentation/logs/clear_full_512_20181005_15H26/mask_rcnn_clear_full_512_0100.h5'

In [ ]:
class InferenceConfig(FishConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# print("Loading weights from ", model_path)
# model.load_weights(model_path, by_name=True)
model.load_weights("/root/data/models/segmentation-2019-02-26.h5")

In [ ]:
model

### 2 - Visualisation of prediction on every image of test set

**Instructions** : Press enter to continue to next image, and ctrl + c or pause the kernel to stop (then enter)

In [ ]:
try:
    while True:
        for image_id in dataset_validate.image_ids:
            original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_validate, inference_config, 
                           image_id, use_mini_mask=False)

            log("original_image", original_image)
            log("image_meta", image_meta)
            log("gt_class_id", gt_class_id)
            log("gt_bbox", gt_bbox)
            log("gt_mask", gt_mask)
            plt.figure(figsize=(10, 10))
            plt.imshow(original_image)
            plt.show()

            visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_validate.class_names, figsize=(8, 8))

            results = model.detect([original_image], verbose=1)
            r = results[0]
            visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_validate.class_names, r['scores'], ax=get_ax())

            visualize.display_differences(original_image, gt_bbox, gt_class_id, gt_mask,
                        r['rois'], r['class_ids'], r['scores'], r['masks'],
                        dataset_validate.class_names)

            input('Press enter to continue: ')
except KeyboardInterrupt:
    pass

### 3 - Evaluation of prediction on every image of test set

In [ ]:
dataset_validate.class_info = [{'id': 0, 'name': 'BG', 'source': ''},
 {'id': 1, 'name': 'salmon', 'source': 'coco'}]

#### 3.1 - mAP

In [ ]:
from mextra.utils import compute_per_class_precision, compute_multiple_per_class_precision

In [ ]:
predictions =\
compute_multiple_per_class_precision(model, inference_config, dataset_validate, iou_threshold=0.5)
complete_predictions = []

for shape in predictions:
    complete_predictions += predictions[shape]
    print("{} ({}): {}".format(shape, len(predictions[shape]), np.mean(predictions[shape])))

print("--------")
print("average: {}".format(np.mean(complete_predictions)))

#### 3.2 - Complementary metrics

In [ ]:
from mextra.utils import compute_per_class_precision
from mrcnn.utils import compute_recall, compute_precision, compute_overlaps
from tqdm import tqdm

In [ ]:
d = {'nb_gt_fish':[], 'nb_pred_fish':[]}
iou_threshold = 0.5

In [ ]:
mean_mask_iou = []
mean_precision = []
mean_recall = []
for i in tqdm(range(len(dataset_validate.image_ids))):
    image_id = dataset_validate.image_ids[i]
    image, _, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_validate, inference_config,
                                image_id, use_mini_mask=False)
    results = model.detect([image], verbose=0)
    r = results[0]
    # Fill histogram dict
    d['nb_gt_fish'].append(float(gt_mask.shape[-1]))
    d['nb_pred_fish'].append(float(r['masks'].shape[-1]))
    # Recall
    mean_recall.append(compute_recall(pred_boxes=r['rois'], gt_boxes=gt_bbox, iou=iou_threshold)[0])
    # Precision
    mean_precision.append(compute_precision(pred_boxes=r['rois'], gt_boxes=gt_bbox, iou=iou_threshold))
    class_precision_info =\
    compute_per_class_precision(gt_bbox, gt_class_id, gt_mask,
                r["rois"], r["class_ids"], r["scores"], r["masks"],
                dataset_validate.class_info, 0.5)
    # Compute mask overlap
    for overlap in list(class_precision_info['salmon']['overlaps']):
        temp = overlap[overlap!=0]
        if len(temp) > 0:
            mean_mask_iou.append(float(temp.max()))

In [ ]:
print('Mean Mask overlap: {}'.format(sum(mean_mask_iou) / len(mean_mask_iou)))
print('Mean precision: {}'.format(sum(mean_precision) / len(mean_precision)))
print('Mean recall: {}'.format(sum(mean_recall) / len(mean_recall)))

In [ ]:
dataset_validate.class_names